# Text Analysis on prince-toronto data

In [1]:
import numpy as np
import pandas as pd
import datetime 
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv(r'D:\論文\PHEME9\Data\CSV\prince-toronto.csv', encoding = 'utf-8', header = None)
df.columns = ['src_tweet_id', 'src_user_id', 'src_tweet','src_date','reply_tweet_id','reply_user_id','reply_tweet','reply_date', 'label']
df.reply_tweet = df.reply_tweet.replace(np.nan,"")

In [3]:
df.columns

Index(['src_tweet_id', 'src_user_id', 'src_tweet', 'src_date',
       'reply_tweet_id', 'reply_user_id', 'reply_tweet', 'reply_date',
       'label'],
      dtype='object')

In [4]:
df

,src_tweet_id,src_user_id,src_tweet,src_date,reply_tweet_id,reply_user_id,reply_tweet,reply_date,label
0,529590949136326656,842564341,"As an aside, Toronto is home to @magpietoronto...",Tue Nov 04 11:08:16 +0000 2014,5.295916e+17,1.538385e+09,"@PrinceMuseum @magpietoronto @3RDEYEGIRL Lol, ...",Tue Nov 04 11:11:03 +0000 2014,0
1,529695075560071168,134586361,Prince loves this city. He fell for the Toront...,Tue Nov 04 18:02:01 +0000 2014,5.296965e+17,6.211173e+07,“@ellhah: Prince loves this city. He fell for ...,Tue Nov 04 18:07:49 +0000 2014,0
2,529695075560071168,134586361,Prince loves this city. He fell for the Toront...,Tue Nov 04 18:02:01 +0000 2014,5.296980e+17,4.464473e+07,@ellhah Prince x Red Drink Boutique invite onl...,Tue Nov 04 18:13:30 +0000 2014,0
3,529755373721632768,1395136699,PRINCE gives Toronto purple balls .....,Tue Nov 04 22:01:38 +0000 2014,NaN,NaN,,NaN,0
4,529761572881985536,43451286,Dammit Prince. Toronto needs a win right now.,Tue Nov 04 22:26:16 +0000 2014,NaN,NaN,,NaN,0
...,...,...,...,...,...,...,...,...,...
763,529830319982407680,526528241,From @DonnaGrantis on Facebook. @3RDEYEGIRL #T...,Wed Nov 05 02:59:26 +0000 2014,NaN,NaN,,NaN,1
764,529830606210093058,291792912,PRINCE in Toronto details. Just a production r...,Wed Nov 05 03:00:34 +0000 2014,5.298316e+17,1.444735e+07,@TheStromboShow And this happened: http://t.co...,Wed Nov 05 03:04:37 +0000 2014,1
765,529830606210093058,291792912,PRINCE in Toronto details. Just a production r...,Wed Nov 05 03:00:34 +0000 2014,5.298334e+17,5.702375e+08,@TheStromboShow \n\nI don't watch music. #fuck...,Wed Nov 05 03:11:49 +0000 2014,1
766,529830606210093058,291792912,PRINCE in Toronto details. Just a production r...,Wed Nov 05 03:00:34 +0000 2014,5.298344e+17,3.787056e+08,@TheStromboShow But @jannarden said there was...,Wed Nov 05 03:15:33 +0000 2014,1


In [5]:
src_tw_df = df.iloc[:,[0,1,2,3,8]].copy()
src_tw_df = src_tw_df.drop_duplicates()
src_tw_df.reset_index(drop=True,inplace=True)
src_tw_df

,src_tweet_id,src_user_id,src_tweet,src_date,label
0,529590949136326656,842564341,"As an aside, Toronto is home to @magpietoronto...",Tue Nov 04 11:08:16 +0000 2014,0
1,529695075560071168,134586361,Prince loves this city. He fell for the Toront...,Tue Nov 04 18:02:01 +0000 2014,0
2,529755373721632768,1395136699,PRINCE gives Toronto purple balls .....,Tue Nov 04 22:01:38 +0000 2014,0
3,529761572881985536,43451286,Dammit Prince. Toronto needs a win right now.,Tue Nov 04 22:26:16 +0000 2014,0
4,529399936434327552,23607802,can someone get us in 2 the secret prince show...,Mon Nov 03 22:29:15 +0000 2014,1
...,...,...,...,...,...
228,529778109290012673,102611790,this nigga Prince got all the unemployed middl...,Tue Nov 04 23:31:58 +0000 2014,1
229,529818929607475201,21137035,Live Nation wants to apologize for any inconve...,Wed Nov 05 02:14:11 +0000 2014,1
230,529828486975066112,20247647,#Toronto - sorry for the confusion today surro...,Wed Nov 05 02:52:09 +0000 2014,1
231,529830319982407680,526528241,From @DonnaGrantis on Facebook. @3RDEYEGIRL #T...,Wed Nov 05 02:59:26 +0000 2014,1


## Prepocessing
- Define function to process text at one time

In [6]:
import pickle

In [7]:
'''Convert Emojis to Words'''

with open(r'D:\論文\PHEME9\Emoji_Dict.p', 'rb') as fp:
    Emoji_Dict = pickle.load(fp)
Emoji_Dict = {v: k for k, v in Emoji_Dict.items()}

def convert_emojis_to_word(text):
    import re
    for emot in Emoji_Dict:
        text = re.sub(r'('+emot+')', "_".join(Emoji_Dict[emot].replace(",","").replace(":","").split()), text)
    return text

In [8]:
def textprocessing(text):
 
    if 'RT' in text.split():      #cut retweet
        rt = text.find('RT')
        text = text[0:rt]  
        
    quo = text.find('“')           #cut quoting part
    quo_end = text.find('”')
    at = text.find('@')
    if quo != -1:                  #if find “
        if quo_end != -1:             # and if there is also ”
            text = text[0: quo:] + text[quo_end+1: :]
        else:
            text = text[0: quo:]
    elif quo_end != -1:
        text = text[0:at:] + text[quo_end+1: : ]
            
   
    text = convert_emojis_to_word(text)   #convert emoji to word
    text = text.lower()           #lower case    
     
    
    text = " ".join([word for word in text.split() if 'http' not in word    #remove url
                                and not word.startswith(('\'@','@','.@'))
                                and word != 'rt'])   
                 
    import re
    def remove_punct(tweet):
        new_words = []
        for word in tweet:
            w = re.sub(r'[^\w\s]','',word) #remove everything except words and space
            w = re.sub(r'_','',w) #remove underscore as well
            w = re.sub('[0-9]+','',w)
            new_words.append(w)

        return new_words
    text = "".join(remove_punct(text))

    from nltk.tokenize import TweetTokenizer
    tknzr = TweetTokenizer(strip_handles=True)
    text = tknzr.tokenize(text)
    
    from nltk.corpus import stopwords
    stop_words=set(stopwords.words("english"))
    stop_words=set([a for a in list(stop_words) if re.search("n't|no|not",a) ==None])
    stop_words.add('now')
    stop_words.add('amp')
    def filterstopwords(tw):
        filter_stopwords = []
        for w in tw:
            if w not in stop_words:
                filter_stopwords.append(w)
        return filter_stopwords
    text = filterstopwords(text)
    
    
    from nltk.stem import WordNetLemmatizer
    lemmatizer = WordNetLemmatizer()
    text = [lemmatizer.lemmatize(i,'v') for i in text]
    
#     from nltk.stem.porter import PorterStemmer
#     stem = PorterStemmer()
#     text = [stem.stem(i) for i in text]
    
    text = " ".join(text)
    
    return text

In [11]:
print("Source tweet: ", df.src_tweet.iloc[527])
print("original text: ", df.reply_tweet.iloc[527])
print("cleaned text: ", textprocessing(df.reply_tweet.iloc[527]))

Source tweet:  LiveNation confirms that Prince will not be performing at Massey Hall tonight.
original text:  “@CP24: LiveNation confirms that Prince will not be performing at Massey Hall tonight.” @AkariBatman HA!😒
cleaned text:  haunamusedface


In [9]:
quotweet = df[df.reply_tweet.str.find('“') != -1].index
quo_tweet = df[df.reply_tweet.str.find('”') != -1].index
rtt = df[df.reply_tweet.str.find('RT') != -1].index
at =  df[df.reply_tweet.str.find('@') != -1].index

set(quo_tweet).difference(set(rtt)).difference(set(quotweet)).difference(set(at))


set()

In [10]:
df[df.reply_tweet.str.find('”') != -1].index

Int64Index([  1,   5,  10,  17,  49,  50,  64,  71,  72,  94,  96,  97, 100,
            101, 103, 107, 111, 113, 115, 116, 117, 121, 125, 126, 128, 135,
            156, 158, 179, 196, 222, 236, 246, 248, 249, 259, 267, 285, 292,
            296, 305, 329, 330, 445, 450, 451, 452, 461, 462, 464, 467, 477,
            496, 515, 519, 527, 547, 553, 554, 645, 649, 663, 664, 681, 693,
            703],
           dtype='int64')

### applied on original data (**df**)

In [12]:
src_tw_df['cleaned_src_tw'] = src_tw_df.src_tweet.apply(textprocessing)
df['cleaned_reply_tw'] = df.reply_tweet.apply(textprocessing)

In [13]:
df.head()

,src_tweet_id,src_user_id,src_tweet,src_date,reply_tweet_id,reply_user_id,reply_tweet,reply_date,label,cleaned_reply_tw
0,529590949136326656,842564341,"As an aside, Toronto is home to @magpietoronto...",Tue Nov 04 11:08:16 +0000 2014,5.295916e+17,1.538385e+09,"@PrinceMuseum @magpietoronto @3RDEYEGIRL Lol, ...",Tue Nov 04 11:11:03 +0000 2014,0,lol hopefully stock summer range pop australia
1,529695075560071168,134586361,Prince loves this city. He fell for the Toront...,Tue Nov 04 18:02:01 +0000 2014,5.296965e+17,6.211173e+07,“@ellhah: Prince loves this city. He fell for ...,Tue Nov 04 18:07:49 +0000 2014,0,
2,529695075560071168,134586361,Prince loves this city. He fell for the Toront...,Tue Nov 04 18:02:01 +0000 2014,5.296980e+17,4.464473e+07,@ellhah Prince x Red Drink Boutique invite onl...,Tue Nov 04 18:13:30 +0000 2014,0,prince x red drink boutique invite party story...
3,529755373721632768,1395136699,PRINCE gives Toronto purple balls .....,Tue Nov 04 22:01:38 +0000 2014,NaN,NaN,,NaN,0,
4,529761572881985536,43451286,Dammit Prince. Toronto needs a win right now.,Tue Nov 04 22:26:16 +0000 2014,NaN,NaN,,NaN,0,


In [14]:
src_tw_df.head()

,src_tweet_id,src_user_id,src_tweet,src_date,label,cleaned_src_tw
0,529590949136326656,842564341,"As an aside, Toronto is home to @magpietoronto...",Tue Nov 04 11:08:16 +0000 2014,0,aside toronto home prince get lot stage clothe
1,529695075560071168,134586361,Prince loves this city. He fell for the Toront...,Tue Nov 04 18:02:01 +0000 2014,0,prince love city fell toronto woman trap happe...
2,529755373721632768,1395136699,PRINCE gives Toronto purple balls .....,Tue Nov 04 22:01:38 +0000 2014,0,prince give toronto purple ball
3,529761572881985536,43451286,Dammit Prince. Toronto needs a win right now.,Tue Nov 04 22:26:16 +0000 2014,0,dammit prince toronto need win right
4,529399936434327552,23607802,can someone get us in 2 the secret prince show...,Mon Nov 03 22:29:15 +0000 2014,1,someone get us secret prince show toronto tomo...


In [15]:
src_tw_df.cleaned_src_tw.iloc[158]

'live nation confirm prince not perform toronto tonight'

## Sentiment Analysis

In [16]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [17]:
def get_sentiment(tweet):
    analyzer = SentimentIntensityAnalyzer()
    vs = analyzer.polarity_scores(tweet)
    if vs['compound'] >=0.05:
        sentiment_label = 'Positive'
    elif (vs['compound'] > -0.05) & (vs['compound'] < 0.05):
        sentiment_label = 'Neutral'
    elif vs['compound']<= -0.05:
        sentiment_label = 'Negative' 
    result = sentiment_label
    return result

### applied on src tweet & reply tweet

In [18]:
src_tw_df['src_sentiment']= src_tw_df.cleaned_src_tw.apply(get_sentiment)
df['reply_sentiment']= df.cleaned_reply_tw.apply(get_sentiment)

In [19]:
df.head(1)

,src_tweet_id,src_user_id,src_tweet,src_date,reply_tweet_id,reply_user_id,reply_tweet,reply_date,label,cleaned_reply_tw,reply_sentiment
0,529590949136326656,842564341,"As an aside, Toronto is home to @magpietoronto...",Tue Nov 04 11:08:16 +0000 2014,5.295916e+17,1.538385e+09,"@PrinceMuseum @magpietoronto @3RDEYEGIRL Lol, ...",Tue Nov 04 11:11:03 +0000 2014,0,lol hopefully stock summer range pop australia,Positive


In [20]:
src_tw_df.head(1)

,src_tweet_id,src_user_id,src_tweet,src_date,label,cleaned_src_tw,src_sentiment
0,529590949136326656,842564341,"As an aside, Toronto is home to @magpietoronto...",Tue Nov 04 11:08:16 +0000 2014,0,aside toronto home prince get lot stage clothe,Neutral


In [21]:
df.to_csv(r'D:\論文\PHEME9\Data\CSV\prince-toronto-df.csv',index=False)

In [22]:
src_tw_df.to_csv(r'D:\論文\PHEME9\Data\CSV\prince-toronto-src.csv',index=False)